In [ ]:
import numpy as np
import torch
from torch.optim import Adam
from sklearn.gaussian_process import GaussianProcessRegressor

import sys
sys.path.append("..")
import moses
from moses.vae import VAE
from moses.vae_property import VAEPROPERTY
from moses.utils import CharVocab, StringDataset, SELFIESVocab
from moses.vae.trainer import VAETrainer
from moses.vae_property.trainer import VAEPROPERTYTrainer 
from tqdm import tqdm


import numpy as np
import pandas as pd
import torch
from rdkit import Chem
from rdkit.Chem import PandasTools
import zipfile
from io import BytesIO

import selfies as sf

### Try out with molecules

In [ ]:
train_df = pd.read_csv("../moses/dataset/data/ZINC/train.csv")
train_df.head(3)

In [ ]:
train_df = train_df[:50000]

In [ ]:
folder_path = "../checkpoints/ZINC_vae_property_20240605_152402"
config = torch.load(f'{folder_path}/vae_property_config.pt')
vocab = torch.load(f'{folder_path}/vae_property_vocab.pt')

print(f"Use Selfies: {config.use_selfies}")

In [ ]:
config.reg_prop_tasks

In [ ]:
model_path = f'{folder_path}/vae_property_model_080.pt'

train_data = moses.get_dataset('train', config)[:50000]

model = VAEPROPERTY(vocab, config)
model.load_state_dict(torch.load(model_path))

trainer = VAEPROPERTYTrainer(config)
sample_loader = trainer.get_dataloader(model, train_data, shuffle=False)

In [ ]:
model.eval()

z_list = []
y_list = []

for step, batch in enumerate(sample_loader):
    if len(batch[0]) == config.n_batch:
        input_batch = tuple(data.to(model.device) for data in batch[0])
        y1 = np.array(train_df.iloc[step*config.n_batch:(step+1)*config.n_batch].SAS)
        y2 = np.array(train_df.iloc[step*config.n_batch:(step+1)*config.n_batch].qed)
        y = 5*y2 + y1
        mu, z, kl_loss = model.forward_encoder(input_batch)
        z = mu.detach().cpu().numpy()
        z_list.extend(z)
        y_list.extend(y)

z_list = np.array(z_list).squeeze()
y_list = np.array(y_list)
# y_list = y_list.squeeze()

In [ ]:
print(z_list.shape)
print(y_list.shape)

In [ ]:
#def f(x):
#    # 임의의 함수 예시 (사용자가 원하는 함수로 대체)
#    return -np.sum((x - 2)**2)

# Numerical gradient 계산 함수
def numerical_gradient(f, clf, x, epsilon=1e-8):
    grad = np.zeros_like(x)
    x_plus = x.copy()
    x_minus = x.copy()
    x_plus += epsilon
    x_minus -= epsilon
    
    grad = (f(x_plus.reshape(1,-1), clf) - f(x_minus.reshape(1,-1), clf)) / (2 * epsilon)
    return grad

# Adam Optimizer for Gradient Ascent
def adam_gradient_ascent(f, clf, initial_x, learning_rate=0.01, max_iter=1000, tolerance=1e-9, beta1=0.9, beta2=0.999, epsilon=1e-8):
    x = initial_x
    m = np.zeros_like(x)  # Initialize the first moment vector
    v = np.zeros_like(x)  # Initialize the second moment vector
    t = 0  # Initialize timestep

    for i in range(max_iter):
        t += 1
        grad = numerical_gradient(f, clf, x)
        
        m = beta1 * m + (1 - beta1) * grad  # Update biased first moment estimate
        v = beta2 * v + (1 - beta2) * (grad ** 2)  # Update biased second moment estimate

        m_hat = m / (1 - beta1 ** t)  # Compute bias-corrected first moment estimate
        v_hat = v / (1 - beta2 ** t)  # Compute bias-corrected second moment estimate

        new_x = x + learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)  # Update the parameters
        
        # 수렴 조건 체크
        if np.linalg.norm(new_x - x) < tolerance:
            break
        x = new_x
       # print(f"Iteration {i+1}: x = {x}, f(x) = {f(x)}")
    
    return x

In [ ]:
GP_train_X = z_list[:1000]
GP_train_y = y_list[:1000]

In [ ]:
rest_z = z_list[1000:1020]
rest_y = y_list[1000:1020]

In [ ]:
clf = GaussianProcessRegressor(random_state=42)
clf.fit(GP_train_X, GP_train_y)

In [ ]:
print(f"R2 Score: {clf.score(rest_z, rest_y)}")
print(f"R2 Score: {clf.score(GP_train_X, GP_train_y)}")

In [ ]:
def f(x, gaussian):
    pred_mean = gaussian.predict(x)[0]

    return pred_mean

In [ ]:
rest_z.shape

In [ ]:
optimal_z_list = []

# Adam Gradient Ascent 수행
for z in tqdm(rest_z):
    initial_point = np.array(z)
    optimal_z = adam_gradient_ascent(f, clf, initial_point)
    optimal_z_list.append(optimal_z)

In [ ]:
np.array(optimal_z_list).shape

In [ ]:
optimal_z_list = torch.tensor(optimal_z_list).float()

In [ ]:
gen_molecule = model.sample(n_batch=len(optimal_z_list), z=optimal_z_list, temp=1.0)

In [ ]:
gen_molecule

In [ ]:
if config.use_selfies:
    
    viz_df = pd.DataFrame({"SELFIES": gen_molecule})

    smiles = []
    for selfies in viz_df['SELFIES']:
        smiles.append(sf.decoder(selfies))

    viz_df['RoMol'] = smiles
    viz_df['RoMol'] = viz_df['RoMol'].apply(Chem.MolFromSmiles)
    display(PandasTools.FrameToGridImage(viz_df, column='RoMol', legendsCol='SELFIES', molsPerRow=4))
else:
    viz_df = pd.DataFrame({"SMILES": gen_molecule})
    viz_df['RoMol'] = viz_df['SMILES'].apply(Chem.MolFromSmiles)
    display(PandasTools.FrameToGridImage(viz_df, column='RoMol', legendsCol='SMILES', molsPerRow=4))